In [1]:
clickhouse_url = 'jdbc:clickhouse://clickhouse-node1:9000/movies'
properties = {
    'user': 'default',
    'password': '',
    'driver': 'com.github.housepower.jdbc.ClickHouseDriver'
}
clickhouse_query='select * from movies.movie_frame'

In [2]:
from pyspark.sql import SparkSession
appName="Connect To clickhouse - via JDBC"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .master('spark://spark-master:7077') \
    .config('spark.executor.extraClassPath","/opt/clickhouse-native-jdbc-shaded-2.6.4.jar') \
    .getOrCreate()
print('AAAA')

AAAA


In [3]:
d = spark.read.jdbc(url=clickhouse_url, table=clickhouse_query, properties=properties)

Py4JJavaError: An error occurred while calling o28.jdbc.
: java.lang.ClassNotFoundException: com.github.housepower.jdbc.ClickHouseDriver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:587)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:520)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRe

In [ ]:
print(d.show(1))